# Testing Dynamic Best Model Selection in edaflow ML Workflow

This notebook demonstrates how to test the dynamic selection of the best model in the edaflow ML workflow, ensuring that the code works for any primary metric (e.g., accuracy, f1, roc_auc).

## Outline & Test Plan

1. Import required libraries and edaflow
2. Load a sample dataset
3. Run the ML workflow with different primary metrics
4. Assert that the best model is selected according to the specified metric
5. Display results

In [ ]:
import edaflow.ml as ml
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np

In [ ]:
# Load sample dataset (breast cancer)
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
def test_best_model_selection(primary_metric):
    # Step 1: Setup ML Experiment
    config = ml.setup_ml_experiment(
        X=X,
        y=y,
        test_size=0.2,
        val_size=0.15,
        experiment_name=f"test_{primary_metric}",
        random_state=42,
        primary_metric=primary_metric
    )
    
    # Step 2: Compare Multiple Models
    models = {
        'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'gradient_boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'logistic_regression': LogisticRegression(random_state=42),
        'svm': SVC(probability=True, random_state=42)
    }
    for name, model in models.items():
        model.fit(config['X_train'], config['y_train'])
    
    # Step 3: Compare Models
    comparison_results = ml.compare_models(
        models=models,
        X_train=config['X_train'],
        y_train=config['y_train'],
        X_test=config['X_test'],
        y_test=config['y_test'],
        cv_folds=5,
        scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    )
    
    # Step 4: Rank Models and Select Best Performer (dynamic)
    ranked_df = ml.rank_models(comparison_results, primary_metric)
    best_model_traditional = ranked_df.iloc[0]['model']
    best_model = ml.rank_models(
        comparison_results,
        primary_metric,
        return_format='list'
    )[0]['model_name']
    
    # Assert both methods agree
    assert best_model == best_model_traditional, f"Mismatch: {best_model} vs {best_model_traditional}"
    
    # Assert the best model has the highest score for the metric
    best_score = ranked_df.iloc[0][primary_metric]
    assert np.isclose(best_score, ranked_df[primary_metric].max()), f"Best score is not max for {primary_metric}"
    
    print(f"✅ Best model for {primary_metric}: {best_model} (score: {best_score:.4f})")

In [ ]:
# Test for different metrics
test_metrics = ['accuracy', 'f1', 'roc_auc']
for metric in test_metrics:
    test_best_model_selection(metric)